# LAB 7: Error analysis

Objectives
* Construct a  linear text classifier using SGDClassifier
* Evaluate its performance and categorize the errors that it makes
* Eaxmine model's coefficients and decision function values
* Interpret model results using LIME

In [1]:
import numpy as np
import pandas as pd
from cytoolz import *
from tqdm.auto import tqdm

tqdm.pandas()

---

## Load data

In [2]:
train = pd.read_parquet(
    "s3://ling583/lab7-train.parquet", storage_options={"anon": True}
)
test = pd.read_parquet("s3://ling583/lab7-test.parquet", storage_options={"anon": True})

In [3]:
import spacy

nlp = spacy.load(
    "en_core_web_sm",
    exclude=["tagger", "parser", "ner", "lemmatizer", "attribute_ruler"],
)


def tokenize(text):
    doc = nlp.tokenizer(text)
    return [t.norm_ for t in doc if not (t.is_space or t.is_punct or t.like_num)]

In [4]:
import multiprocessing as mp

In [5]:
with mp.Pool() as p:
    train["tokens"] = pd.Series(p.imap(tokenize, tqdm(train["text"]), chunksize=100))
    test["tokens"] = pd.Series(p.imap(tokenize, tqdm(test["text"]), chunksize=100))

  0%|          | 0/19054 [00:00<?, ?it/s]

  0%|          | 0/4764 [00:00<?, ?it/s]

The labels are: GPOL = domestic politics, GSPO = sports, GVIO = war/civil war, GJOB = labor issues

In [6]:
train["topics"].value_counts()

GPOL    7410
GSPO    5639
GVIO    3712
GJOB    2293
Name: topics, dtype: int64

---

## Baseline classifier

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.pipeline import make_pipeline

In [8]:
baseline = make_pipeline(CountVectorizer(analyzer=identity), SGDClassifier())
baseline.fit(train["tokens"], train["topics"])
base_predicted = baseline.predict(test["tokens"])
print(classification_report(test["topics"], base_predicted))

              precision    recall  f1-score   support

        GJOB       0.97      0.92      0.94       573
        GPOL       0.94      0.95      0.94      1853
        GSPO       1.00      0.99      0.99      1410
        GVIO       0.90      0.91      0.90       928

    accuracy                           0.95      4764
   macro avg       0.95      0.94      0.95      4764
weighted avg       0.95      0.95      0.95      4764



----

## Hyperparameter search

Find an optimal set of hyperparameters for a Tfidf+SGDClassifier model

In [9]:
import mlflow
from dask_ml.model_selection import RandomizedSearchCV
from logger import log_search
from scipy.stats.distributions import loguniform, randint, uniform

In [10]:
from warnings import simplefilter

simplefilter(action="ignore", category=FutureWarning)

In [11]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:34173")
client

Client Scheduler: tcp://127.0.0.1:34173 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.62 GB


In [12]:
mlflow.set_experiment("lab-7")
sgd = make_pipeline(
    CountVectorizer(analyzer=identity), TfidfTransformer(), SGDClassifier()
)

INFO: 'lab-7' does not exist. Creating a new experiment


In [13]:
%%time

search = RandomizedSearchCV(
    sgd,
    {
        "countvectorizer__min_df": randint(1, 20),
        "countvectorizer__max_df": uniform(0.5, 0.5),
        "tfidftransformer__use_idf": [True, False],
        "sgdclassifier__alpha": loguniform(1e-6, 1e-2),
    },
    n_iter=50,
    scoring="f1_macro",
)
search.fit(train["tokens"], train["topics"])
log_search(search)

CPU times: user 10.1 s, sys: 1.44 s, total: 11.5 s
Wall time: 3min 29s


---

## Compare optimized model to baseline

In [15]:
sgd = make_pipeline(
    CountVectorizer(analyzer=identity, min_df=5, max_df=0.75),
    TfidfTransformer(use_idf=True),
    SGDClassifier(alpha=1e-4),
)
sgd.fit(train["tokens"], train["topics"])
predicted = sgd.predict(test["tokens"])
print(classification_report(test["topics"], predicted))

              precision    recall  f1-score   support

        GJOB       0.97      0.94      0.95       573
        GPOL       0.94      0.97      0.95      1853
        GSPO       1.00      0.99      1.00      1410
        GVIO       0.92      0.90      0.91       928

    accuracy                           0.96      4764
   macro avg       0.96      0.95      0.95      4764
weighted avg       0.96      0.96      0.96      4764



In [16]:
base_f1 = f1_score(test["topics"], base_predicted, average="macro")
sgd_f1 = f1_score(test["topics"], predicted, average="macro")

In [17]:
base_f1, sgd_f1, sgd_f1 - base_f1

(0.9463167933227535, 0.9543078380635164, 0.007991044740762843)

In [18]:
(sgd_f1 - base_f1) / (1 - base_f1)

0.1488555776633558

In [19]:
from scipy.stats import binom_test, wilcoxon

In [20]:
diff = (predicted == test["topics"]).astype(int) - (
    base_predicted == test["topics"]
).astype(int)
sum(diff == 1), sum(diff == -1), sum(diff == 0)

(81, 44, 4639)

In [21]:
binom_test([sum(diff == 1), sum(diff == -1)], alternative="greater")

0.0005962486102612331

In [22]:
wilcoxon(diff, alternative="greater")

WilcoxonResult(statistic=5103.0, pvalue=0.00046751317631981267)

**TO DO:** Summarize your results: how much better is the optimized model? Is it significantly better than the baseline?

The optimized model is much better than the baseline by a huge amount, as the macro average f1-score for the optimized model is 0.03 greater than the baselines, which is a significant amount, especially compared to the video where the difference was 0.01. In addition, the binomial and wilcoxon test p-values are very small, indicating that there is a difference, assuming the alpha is 0.05. Also the p-values were much smaller than ones in the video as well. 

-----

## Save model

In [23]:
import cloudpickle

In [25]:
sgd = make_pipeline(
    CountVectorizer(preprocessor=identity, tokenizer=tokenize, min_df=5, max_df=0.75),
    TfidfTransformer(use_idf=True),
    SGDClassifier(alpha=1e-4),
)
sgd.fit(train["text"], train["topics"])
predicted = sgd.predict(test["text"])
print(classification_report(test["topics"], predicted))

              precision    recall  f1-score   support

        GJOB       0.98      0.93      0.95       573
        GPOL       0.94      0.96      0.95      1853
        GSPO       1.00      0.99      1.00      1410
        GVIO       0.92      0.91      0.92       928

    accuracy                           0.96      4764
   macro avg       0.96      0.95      0.95      4764
weighted avg       0.96      0.96      0.96      4764



In [26]:
cloudpickle.dump(sgd, open("sgd.model", "wb"))